# Data analysis using hw6_library

In [1]:
# Installing the library:

#!git clone
!git clone https://github.com/danidlsa/comp4ds_hw6.git

Clonando en 'comp4ds_hw6'...
remote: Enumerating objects: 177, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 177 (delta 10), reused 16 (delta 3), pack-reused 138
Recibiendo objetos: 100% (177/177), 594.70 KiB | 2.63 MiB/s, listo.
Resolviendo deltas: 100% (74/74), listo.


In [2]:
#install from the cloned git folder
!pip install -e comp4ds_hw6

Obtaining file:///home/danidlsa-u/Documentos/comp4ds_hw6
  Preparing metadata (setup.py) ... done
  Running setup.py develop for hw6-library


## Load data and split data frame

In [31]:
from hw6_library.DataLoad import create_data as data

In [32]:
diabetes_data = data.create_data() #instantiate create_data class

In [33]:
diabetes_data.df # showing df attribute

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,...,ventilated_apache,wbc_apache,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
0,0,214826,118,68.0,22.732803,0,Caucasian,M,180.30,Floor,...,0,14.1,0,0,0,0,0,0,0,1
1,1,246060,81,77.0,27.421875,0,Caucasian,F,160.00,Floor,...,1,12.7,0,0,0,0,0,0,0,1
2,2,276985,118,25.0,31.952749,0,Caucasian,F,172.70,Emergency Department,...,0,NaN,0,0,0,0,0,0,0,0
3,3,262220,118,81.0,22.635548,1,Caucasian,F,165.10,Operating Room,...,1,8.0,0,0,0,0,0,0,0,0
4,4,201746,33,19.0,NaN,0,Caucasian,M,188.00,NaN,...,0,NaN,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,167042,140,NaN,51.952814,0,Caucasian,F,137.20,Floor,...,0,7.0,0,0,0,0,0,0,0,0
9996,9996,165009,89,34.0,26.110113,0,Caucasian,F,175.26,Emergency Department,...,0,9.5,0,0,0,0,0,0,0,0
9997,9997,234052,137,30.0,23.774241,0,Native American,F,157.48,Emergency Department,...,0,NaN,0,0,0,0,0,0,0,0
9998,9998,249290,89,67.0,31.330708,0,Caucasian,F,157.48,Direct Admit,...,0,6.4,0,0,0,0,0,0,0,0


In [34]:
diabetes_train, diabetes_test = diabetes_data.split(0.8) # Using split method

print("Shape of train df: " + str(diabetes_train.shape))
print("Shape of test df: " + str(diabetes_test.shape))


Shape of train df: (8000, 53)
Shape of test df: (2000, 53)


## Preprocess data

In [35]:
from hw6_library.Preprocessing import Preprocessing as pp

#### Clean demographics 

Preprocessor class that removes those rows that contain NaN values in the columns: age, gender, ethnicity.

In [36]:
# Instanciating class with train data
df_train_clean1 = pp.clean_demographics(diabetes_train) 
# Applying method
df_train_clean1= df_train_clean1.drop_selected_nans()        
        
# Instantiating class with test data
df_test_clean1 = pp.clean_demographics(diabetes_test)        
# Applying method
df_test_clean1= df_test_clean1.drop_selected_nans()        


In [37]:
vars = ['age', 'gender', 'ethnicity']
for v in vars:
    print("Number of NaNs for " + v + ' in train set:' + str(df_train_clean1[v].isna().sum()))

for v in vars:
    print("Number of NaNs for " + v + ' in test set:' + str(df_test_clean1[v].isna().sum()))


Number of NaNs for age in train set:0
Number of NaNs for gender in train set:0
Number of NaNs for ethnicity in train set:0
Number of NaNs for age in test set:0
Number of NaNs for gender in test set:0
Number of NaNs for ethnicity in test set:0


### Clean measurements

Preprocessor class that fills NaN with the mean value of the column in the columns: height, weight.

In [38]:
df_train_clean2 = pp.clean_measurements(df_train_clean1)
vars_to_fill = df_train_clean2.vars_to_fill
df_train_clean2 = df_train_clean2.fill_nans()

df_test_clean2 = pp.clean_measurements(df_test_clean1).fill_nans()


In [39]:
for v in vars_to_fill:
    print("Number of NaNs for " + v + ' in train set:' + str(df_train_clean2[v].isna().sum()))
    
for v in vars_to_fill:
    print("Number of NaNs for " + v + ' in test set:' + str(df_test_clean2[v].isna().sum()))

Number of NaNs for height in train set:0
Number of NaNs for weight in train set:0
Number of NaNs for height in test set:0
Number of NaNs for weight in test set:0


## Feature creation

Creating two feature classes: gender and ethnicity

In [40]:
from hw6_library.FeatureCreation import Transform_features as tf

In [41]:
# Instanciating class and applying ethnicity one hot encoding
df_train_eth = tf.ethnicity_oh(df_train_clean2).one_hot_enc()

df_test_eth= tf.ethnicity_oh(df_test_clean2).one_hot_enc()


In [42]:
# Instanciating class and applying gender one hot encoding. The difference: this variation drops first value
# The one before (etnicity_oh) keeps all values

df_train_gender= tf.gender_oh(df_train_eth).one_hot_enc()



df_test_gender= tf.gender_oh(df_train_eth).one_hot_enc()


In [43]:
df_train_gender.head()

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,...,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus,African American,Asian,Caucasian,Hispanic,Native American,Other/Unknown,M
8856,8856,278720,168,76.0,32.120596,0,Caucasian,F,154.94,Direct Admit,...,0,0,0,0,0,1,0,0,0,0
1545,1545,204405,149,71.0,NaN,0,Caucasian,F,154.90,Direct Admit,...,0,0,0,0,0,1,0,0,0,0
8136,8136,215017,137,86.0,22.172200,0,Caucasian,F,154.90,NaN,...,0,0,0,0,0,1,0,0,0,0
7768,7768,267005,83,55.0,38.583426,1,Caucasian,M,170.30,Operating Room,...,0,0,0,0,0,1,0,0,0,1
7782,7782,165279,118,51.0,20.845960,0,Caucasian,M,177.80,Emergency Department,...,0,0,0,0,0,1,0,0,0,1


In [44]:
df_test_gender.head()

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,...,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus,African American,Asian,Caucasian,Hispanic,Native American,Other/Unknown,M
8856,8856,278720,168,76.0,32.120596,0,Caucasian,F,154.94,Direct Admit,...,0,0,0,0,0,1,0,0,0,0
1545,1545,204405,149,71.0,NaN,0,Caucasian,F,154.90,Direct Admit,...,0,0,0,0,0,1,0,0,0,0
8136,8136,215017,137,86.0,22.172200,0,Caucasian,F,154.90,NaN,...,0,0,0,0,0,1,0,0,0,0
7768,7768,267005,83,55.0,38.583426,1,Caucasian,M,170.30,Operating Room,...,0,0,0,0,0,1,0,0,0,1
7782,7782,165279,118,51.0,20.845960,0,Caucasian,M,177.80,Emergency Department,...,0,0,0,0,0,1,0,0,0,1


## Modeling

In [45]:
from hw6_library.Modeling import data_modeling as dm

# This module will allow us to instanciate a class that trains and predicts values using a Random Forest model

In [46]:
# First, we select a list of features

features = ['age','height','weight','aids','cirrhosis', 'hepatic_failure', 'immunosuppression', 'leukemia', 'lymphoma', 'solid_tumor_with_metastasis',
            'M', 'Asian', 'Hispanic', 'Native American', 'African American' ]



In [47]:
# Second, we instanciate the class, using as input: our list of features, the name of our target variable,
# and two hyperparameters for the model: the number of estimators and the max_depth for the decision trees

model_randomf = dm.Train_and_predict_RF(features, "diabetes_mellitus", 8, 4)



In [48]:
# We can check the attributes of this model

# We have one public attribute: model
print("Model: " + str(model_randomf.model))

# And two private attributes: features and target

print("Features: " + str(model_randomf._Train_and_predict_RF__features))

print("Target: " + str(model_randomf._Train_and_predict_RF__target))

Model: RandomForestClassifier(max_depth=4, n_estimators=8)
Features: ['age', 'height', 'weight', 'aids', 'cirrhosis', 'hepatic_failure', 'immunosuppression', 'leukemia', 'lymphoma', 'solid_tumor_with_metastasis', 'M', 'Asian', 'Hispanic', 'Native American', 'African American']
Target: diabetes_mellitus


In [49]:
# Third, through the method 'train', we train the model, using as input our pre-processed train dataset
model_randomf.train(df_train_gender)



/home/danidlsa-u/Documentos/comp4ds_hw6/src/hw6_library/Modeling/data_modeling.py:38: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.model.fit(self.X, self.y)


In [50]:
# Lastly, we can obtain an array of predicted values for diabetes_mellitus

predicted_values= model_randomf.predict(df_test_gender)
print(predicted_values)

[0 0 0 ... 0 0 0]


In [51]:
# In order to assess the accuracy of the model, we will compare this results with our original y data in the test set

# To do this, we include the predicted_values array into our test dataset:
df_test_gender["ypred"] = predicted_values
df_test_gender[["diabetes_mellitus", "ypred"]].head()

,diabetes_mellitus,ypred
8856,0,0
1545,0,0
8136,0,0
7768,0,0
7782,0,0


In [52]:
# To evaluate accuracy, we can get the ROC AUC score. The results show this is not a very good model, 
# so we should continue working on it!

from sklearn.metrics import roc_auc_score

roc_auc_score(df_test_gender["diabetes_mellitus"], df_test_gender["ypred"])

0.5025655644241733